In [1]:
import os
import config

os.environ["OPENAI_API_KEY"] = config.OPENAI_API_KEY

In [ ]:
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import DirectoryLoader, TextLoader
from langchain.chains import RetrievalQA
from langchain.memory import ConversationBufferMemory, ReadOnlySharedMemory
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.tools import DuckDuckGoSearchRun, BaseTool
from langchain.agents import initialize_agent, Tool, AgentType, AgentExecutor, ZeroShotAgent
from langchain import hub, LLMMathChain, OpenAI, LLMChain, PromptTemplate

In [ ]:
# Ruta al directorio que contiene la base de datos de vectores
persist_directory = 'db'

In [ ]:
# Cargar la base de datos de vectores existente
vectordb = Chroma(persist_directory=persist_directory, embedding_function=OpenAIEmbeddings())

In [ ]:
# Configuración del modelo de lenguaje
llm = ChatOpenAI(temperature=0, max_tokens=1000, model="gpt-3.5-turbo")

In [ ]:
# Configuración de la memoria
memory = ConversationBufferMemory(memory_key="chat_history")
readonlymemory = ReadOnlySharedMemory(memory=memory)

In [ ]:
# Configuración del retriever para QA
retriever = vectordb.as_retriever(search_kwargs={"k": 2})
qa_chain = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=retriever, memory=readonlymemory)

In [ ]:
tools = [
    Tool(name="QA System", func=qa_chain.run, description="Útil cuando necesitas responder preguntas sobre El mapa curricular de IDeIO."),
    Tool(name="Backup QA", func=DuckDuckGoSearchRun().run, description="Útil cuando necesitas responder preguntas sobre El mapa curricular de IDeIO, pero solo cuando el Sistema de Preguntas y Respuestas de El mapa curricular de IDeIO no pudo responder la consulta."),
    Tool(name="Calculadora", func=LLMMathChain.from_llm(llm=llm, verbose=True).run, description="Herramienta que suma las materias y suma los creditos de las materias"),
]

In [ ]:
prefix = "Inicia una conversación con un humano, respondiendo las siguientes preguntas lo mejor que puedas. Tienes acceso a las siguientes herramientas:"
suffix = "¡Comienza!\n{chat_history}\nPregunta: {input}\n{agent_scratchpad}"

In [ ]:
prompt = ZeroShotAgent.create_prompt(tools, prefix=prefix, suffix=suffix, input_variables=["input", "chat_history", "agent_scratchpad"])
llm_chain = LLMChain(llm=llm, prompt=prompt)
agent = ZeroShotAgent(llm_chain=llm_chain, tools=tools, verbose=True)
agent_chain = AgentExecutor.from_agent_and_tools(agent=agent, tools=tools, verbose=True, memory=memory)

In [ ]:
agent_chain.run(input="¿Qué materia debo cargar antes de Aprendizaje Estadistico?")



> Entering new AgentExecutor chain...
Thought: Necesito saber cuál es la materia que se debe cursar antes de Aprendizaje Estadístico en el mapa curricular de IDeIO.
Action: QA System
Action Input: Pre-requisitos de Aprendizaje Estadístico
Observation: Los pre-requisitos de Aprendizaje Estadístico son cargar y aprobar primero "Estadística Analítica".
Thought:Ya tengo la respuesta final.
Final Answer: Debes cargar y aprobar primero la materia "Estadística Analítica" antes de Aprendizaje Estadístico.

> Finished chain.


'Debes cargar y aprobar primero la materia "Estadística Analítica" antes de Aprendizaje Estadístico.'